Import the necessary libraries

In [26]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import io

Get HTML document behind the URL of the Wikipedia page

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",'html.parser')

Parse and process the html document, extracting the table and the td tags, define list of postcodes, boroughs, neighborhoods

In [3]:
soup = BeautifulSoup(r.text[:], 'html.parser')

In [4]:
tag = soup.table

In [5]:
tables = soup.table

In [6]:
tds = tables.find_all('td')

In [7]:
postcode = tds[0::3]
postcode = list(map(lambda i:str(i)[4:7],postcode))


In [8]:
borough = tds[1::3]
borough = list(map(lambda i:str(i).split('>')[2].split('<')[0], borough))


In [9]:
neighborhood = tds[2::3]
neighborhood = list(map(lambda i:str(i).split('>')[2].split('<')[0], neighborhood))


Process the lists into a Dataframe, organize column order, delete rows with N/A borough, copy borough name for rows with N/A neighborhood

In [186]:

df = pd.DataFrame({'PostalCode':postcode, 'Borough':borough, 'Neighborhood':neighborhood} )
df = df.drop(df[df['Borough']==''].index)
mask = df['Neighborhood'] == ''
df.loc[mask, 'Neighborhood'] = df.loc[mask,'Borough']
df.reset_index(drop = True, inplace = True)
cols = df.columns.tolist()
cols = [cols[2], cols[0], cols[1]]
df = df[cols]
#Combine neighborhoods with same PostalCodes
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].unique().reset_index()#apply(', '.join).reset_index()

In [187]:
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Scarborough, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Scarborough, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough"
9,M1N,Scarborough,"Birch Cliff, Scarborough"


Get the shape of the dataframe

In [178]:
df.shape

(100, 3)

In [179]:
df1 = pd.read_csv('https://cocl.us/Geospatial_data')

In [180]:
pd.set_option('display.max_rows', 110)

In [181]:
df2 = df1.drop([73,74,84]).reset_index()
df2['PostalCode'] = df2['Postal Code']

In [182]:
df2.drop(columns = ['Postal Code', 'index'], inplace = True)

In [183]:
cols = ['PostalCode', 'Latitude', 'Longitude']
df2 = df2[cols]
df3 = df.merge(df2, on = 'PostalCode')

In [184]:
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Scarborough, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[Scarborough, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Scarborough]",43.692657,-79.264848
